In [60]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, create_schema

Connect to postgres

In [61]:
storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)

INFO:storage.py:42: connecting to host: 'localhost', port: 5433, dbname: 'ner_test', user: 'postgres'
INFO:storage.py:58: schema: 'my_schema', temporary: False, role: 'postgres'


In [62]:
storage

In [63]:
del storage['ner_texts']

In [64]:
storage['ner_texts'].create()

INFO:collection.py:92: new empty collection 'ner_texts' created


In [65]:
collection = storage['ner_texts']

In [66]:
storage.conn.commit()

In [67]:
storage.conn.autocommit = False

Read in texts

In [68]:
from estnltk.corpus_processing.parse_koondkorpus import parse_tei_corpus

epltexts = parse_tei_corpus(r'C:\Users\rasmu\Downloads\2001_tei\2001_tei\EPL_2001_01_11.tei', target=['artikkel'], encoding="UTF-8")

In [69]:
#add texts to collection if haven't done it yet
with collection.insert() as collection_insert:
    for text in epltexts:
        text.tag_layer()
        collection_insert(text)
storage.conn.commit()

INFO:collection_text_object_inserter.py:107: inserted 344 texts into the collection 'ner_texts'


In [70]:
from estnltk.taggers.standard.ner.ner_tagger import NerTagger
ner_tagger = NerTagger()

In [71]:
with storage.conn.cursor() as c:
    storage['ner_texts'].add_layer(ner_tagger.get_layer_template())

INFO:collection.py:819: detached layer 'ner' created from template


In [72]:
collection.selected_layers = ['ner']

In [73]:
import multiprocessing

from estnltk.storage import PostgresStorage

def ner_thread(block):
    print("Thread start " + str(block))
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    ner_tagger = NerTagger()
    collection.create_layer_block(tagger=ner_tagger, block=(6,block))


def driver_func():
    processes = 6
    procs = []
    for i in range(processes):
        proc = multiprocessing.Process(target=ner_thread, args=(i,))
        procs.append(proc)
        proc.start()

    for proc in procs:
        proc.join()

In [75]:
if __name__ == '__main__':
    a = driver_func()

In [ ]:
'''n=4
for i in range(n):
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, meta=None, block=(n, i))'''

In [ ]:
'''collection = storage['ner_texts']
collection.create_layer_block(tagger=ner_tagger, meta=None, block=(2, 0))'''

In [ ]:
'''storage_b = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)'''

In [ ]:
'''collection_b = storage_b['ner_texts']
collection_b.create_layer_block(tagger=ner_tagger, meta=None, block=(2, 1))'''

In [ ]:
'''
entity_map = {}
#create a dictionary where keys are tuples of words and values are the nertags assigned to that span

for text in collection:
    for match in text.ner:
        current = entity_map.get(tuple(match.text),[])
        if match.nertag not in current:
            current.append(match.nertag)
            entity_map[tuple(match.text)] = current
'''

In [ ]:
'''import pickle 

#since the first part is unlikely to change later but takes a lot of time, here is a checkpoint where the entity map
#is saved to a separate file so that editing the notebook later can start from here
with open('entity_map.pkl', 'wb') as f:
    pickle.dump(entity_map, f)'''

In [27]:
'''import threading
from multiprocessing import Process

def thread_0():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 0))
    print(threading.current_thread().getName(), 'Exiting')


def thread_1():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 1))
    print(threading.current_thread().getName(), 'Exiting')
    
def thread_2():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 2))
    print(threading.current_thread().getName(), 'Exiting')
    
def thread_3():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 3))
    print(threading.current_thread().getName(), 'Exiting')
    
def thread_4():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 4))
    print(threading.current_thread().getName(), 'Exiting')
    
def thread_5():
    print(threading.current_thread().getName(), 'Starting')
    storage = PostgresStorage(host='localhost',
                          port=5433,
                          dbname='ner_test',
                          user='postgres',
                          password='dbpass',
                          pgpass_file='~/.pgpass',
                          schema='my_schema',
                          role=None,
                          temporary=False)
    collection = storage['ner_texts']
    collection.create_layer_block(tagger=ner_tagger, block=(6, 5))
    print(threading.current_thread().getName(), 'Exiting')

if __name__ == "__main__":
t0 = threading.Thread(name='thread_0', target=thread_0)
t1 = threading.Thread(name='thread_1', target=thread_1)
t2 = threading.Thread(name='thread_2', target=thread_2)
t3 = threading.Thread(name='thread_3', target=thread_3)
t4 = threading.Thread(name='thread_4', target=thread_4)
t5 = threading.Thread(name='thread_5', target=thread_5)

t0.start()
t1.start()
t2.start()
t3.start()
t4.start()
t5.start()'''

In [76]:
collection.selected_layers = ['words','ner']

In [78]:
collection[0]

KeyError: 'Index 0 is outside of the collection'